## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [ ]:
file_id = '1kmXAzIgGI0Yg6HM00pSL3VSXhAYidI6l'
download_file(file_id, 'cool_roof_yolo_labels.zip')
file_id = '1kisSOoULdWoLLo3Aoyel8jowar5gxKlN'
download_file(file_id, 'cool_roof_images.zip')

cool_roof_yolo_labels.zip 파일이 이미 존재합니다.


Downloading...
From (original): https://drive.google.com/uc?id=1kisSOoULdWoLLo3Aoyel8jowar5gxKlN
From (redirected): https://drive.google.com/uc?id=1kisSOoULdWoLLo3Aoyel8jowar5gxKlN&confirm=t&uuid=3dbf20c3-3da7-4360-8d0a-b55067e52b14
To: /content/cool_roof_images.zip
100%|██████████| 216M/216M [00:02<00:00, 96.5MB/s]


In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [ ]:
dataset_extract('cool_roof_yolo_labels.zip')
dataset_extract('cool_roof_images.zip')

Extracting: 100%|██████████| 200/200 [00:02<00:00, 98.80files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
#test,train,valid 폴더 생성
import os
path = '/content/cool_roof/'

for folder in ['train','test','valid']:
    os.makedirs(path+folder+'/images',exist_ok=True)
    os.makedirs(path+folder+'/labels',exist_ok=True)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split

image_path = glob('/content/cool_roof_images/*')
len(image_path)

200

In [47]:
train_path, test_img,_, _ = train_test_split(image_path,image_path,test_size=0.2,random_state=2024)
train_img, valid_img,_, _ = train_test_split(train_path,train_path,test_size=0.2,random_state=2024)
len(train_img),len(valid_img), len(test_img)

(128, 32, 40)

In [48]:
train_img[1].split('/')[-1].replace('jpg','txt')

'roof (138).txt'

In [ ]:
train_img[1].replace('cool_roof_images','cool_roof_yolo_labels').replace('jpg','txt')

'/content/cool_roof_yolo_labels/roof (138).txt'

In [ ]:
# train 파일 이동
for img_path in train_img:
    label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1].replace('jpg','txt')
    if os.path.exists(img_path):
        shutil.move(img_path,path+'train/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'train/labels')
# valid 파일 이동
for img_path in valid_img:
    label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1].replace('jpg','txt')
    if os.path.exists(img_path):
        shutil.move(img_path,path+'valid/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'valid/labels/')
# test 파일 이동
for img_path in test_img:
    label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1].replace('jpg','txt')
    if os.path.exists(img_path):
        shutil.move(img_path,path+'test/images/')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'test/labels/')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
import yaml

yaml_data = {'train' : '/content/cool_roof/train/images',
               'val' : '/content/cool_roof/valid/images',
               'test' : '/content/cool_roof/test/images',
               'names' : ['cool roof', 'generic roof'],
               'nc' : 2 }


yaml_file_path = "/content/cool_roof/data.yaml"
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import settings, YOLO

In [ ]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [43]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [49]:
model_transfer = YOLO(model='yolov8n.pt', task='detect')

100%|██████████| 6.23M/6.23M [00:00<00:00, 84.0MB/s]


In [50]:
model_transfer.train(data='/content/cool_roof/data.yaml',
                    epochs=30,
                    patience=5,
                    seed=2024,
                    pretrained=True,
                    )

Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/cool_roof/data.yaml, epochs=30, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 16.5MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/cool_roof/train/labels... 128 images, 0 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 511.48it/s]

train: New cache created: /content/cool_roof/train/labels.cache


...

                   all         32        239       0.38       0.44      0.384      0.228
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



8 epochs completed in 0.281 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:10<00:00, 10.71s/it]


                   all         32        239      0.918      0.348      0.457      0.285
             cool roof         32         38          1          0      0.118     0.0739
          generic roof         32        201      0.836      0.697      0.796      0.496
Speed: 2.5ms preprocess, 275.4ms inference, 0.0ms loss, 14.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d82807aa4d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [51]:
image_path = '/content/cool_roof/test/images'

model_transfer.predict(source=image_path, save=True)


image 1/40 /content/cool_roof/test/images/roof (1).jpg: 640x640 (no detections), 265.4ms
image 2/40 /content/cool_roof/test/images/roof (107).jpg: 352x640 (no detections), 182.6ms
image 3/40 /content/cool_roof/test/images/roof (109).jpg: 384x640 (no detections), 228.5ms
image 4/40 /content/cool_roof/test/images/roof (110).jpg: 480x640 (no detections), 294.2ms
image 5/40 /content/cool_roof/test/images/roof (114).jpg: 480x640 1 generic roof, 258.5ms
image 6/40 /content/cool_roof/test/images/roof (115).jpg: 480x640 (no detections), 278.6ms
image 7/40 /content/cool_roof/test/images/roof (116).jpg: 480x640 1 generic roof, 278.1ms
image 8/40 /content/cool_roof/test/images/roof (118).jpg: 640x576 1 generic roof, 328.3ms
image 9/40 /content/cool_roof/test/images/roof (12).jpg: 640x608 (no detections), 235.3ms
image 10/40 /content/cool_roof/test/images/roof (120).jpg: 640x576 (no detections), 207.1ms
image 11/40 /content/cool_roof/test/images/roof (122).jpg: 576x640 1 generic roof, 216.9ms
ima

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'cool roof', 1: 'generic roof'}
 obb: None
 orig_img: array([[[ 57,  46,  32],
         [ 56,  44,  32],
         [ 57,  46,  32],
         ...,
         [ 26,  17,   8],
         [ 27,  17,   7],
         [ 27,  18,   5]],
 
        [[ 59,  46,  32],
         [ 54,  42,  30],
         [ 58,  45,  31],
         ...,
         [ 26,  17,   8],
         [ 27,  17,   7],
         [ 27,  18,   5]],
 
        [[ 59,  46,  32],
         [ 58,  45,  31],
         [ 58,  47,  33],
         ...,
         [ 26,  17,   8],
         [ 27,  18,   5],
         [ 28,  19,   6]],
 
        ...,
 
        [[118, 132, 126],
         [124, 136, 130],
         [129, 144, 136],
         ...,
         [114, 130, 123],
         [115, 130, 126],
         [116, 131, 127]],
 
        [[114, 129, 121],
         [116, 130, 124],
         [121, 136, 128],
         .